# Exercise I: Profiling the Julia Set Code
## Task 1.1 Calculate the Clock Granularity of different Python Timers

I run 100 times and caculate the average. The clock granularity is listed following 

time.time(): 7.152557373046875e-07 s = 715.26 ns

timeit: 1.224899997032125e-07 s = 122.49 ns

time.time_ns(): 768.0 ns

In [1]:
import numpy as np
def checktick(time_func):
   M = 200
   timesfound = np.empty((M,))
   for i in range(M):
      t1 =  time_func() # get timestamp from timer
      t2 = time_func() # get timestamp from timer
      while (t2 - t1) < 1e-16: # if zero then we are below clock granularity, retake timing
          t2 = time_func() # get timestamp from timer
      t1 = t2 # this is outside the loop
      timesfound[i] = t1 # record the time stamp
   minDelta = 1000000
   Delta = np.diff(timesfound) # it should be cast to int only when needed
   minDelta = Delta.min()
   return minDelta

In [2]:
import time
results = [checktick(time.time) for _ in range(100)]
average_tick = np.mean(results)
print("The clock granularity of time.time(): ",average_tick * (1e9), "ns")

The clock granularity of time.time():  715.2557373046875 ns


In [3]:
from timeit import default_timer as timer
results = [checktick(timer) for _ in range(100)]
average_tick = np.mean(results)
print("The clock granularity of timeit() ",average_tick* (1e9), "ns")

The clock granularity of timeit()  112.87999993392361 ns


In [4]:
import time
results = [checktick(time.time_ns) for _ in range(100)]
average_tick = np.mean(results)
print("The clock granularity of time.time_ns() ",average_tick, "ns")

The clock granularity of time.time_ns()  768.0 ns


## Task 1.2 Timing the Julia set code functions. 

The best timer is timeit
The average of execution time in calc_pure_python:  0.9094791666999998 s

The standard deviation of execution time in calc_pure_python:  0.02674339780011118 s

The average of execution time in calculate_z_serial_purepython:  0.6889902166999997 s

The standard deviation of execution time in calculate_z_serial_purepython:  0.017586994181006566 s

The value of standard deviation is greater than the clock granularity 

In [5]:
import JuliaSet

In [6]:
calculate_z_serial_purepython=[]
calc_pure_python=[]
for _ in range(10):
    time1,time2=JuliaSet.calc_pure_python(desired_width=1000, max_iterations=30)
    calculate_z_serial_purepython.append(time1)
    calc_pure_python.append(time2)
    

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.668167333 seconds
@timefn: calc_pure_python took 0.8915752079999999 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.6748907500000001 seconds
@timefn: calc_pure_python took 0.8858226670000002 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.6728489579999999 seconds
@timefn: calc_pure_python took 0.8840081250000003 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.6732088750000003 seconds
@timefn: calc_pure_python took 0.8860660829999998 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.6669616249999999 seconds
@timefn: calc_pure_python took 0.8767898339999993 seconds
Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 0.6648984169999999 seconds
@timefn: calc_pure_python took 0.875557

In [7]:
import statistics
print("The average execution time of calc_pure_python: ",statistics.mean(calc_pure_python) )
print("The standard dev execution time of calc_pure_python: ",statistics.stdev(calc_pure_python) )
print("The average execution time of calculate_z_serial_purepython: ",statistics.mean(calculate_z_serial_purepython) )
print("The standard dev execution time of calculate_z_serial_purepython: ",statistics.stdev(calculate_z_serial_purepython) )

The average execution time of calc_pure_python:  0.8940317040999999
The standard dev execution time of calc_pure_python:  0.020271528353383916
The average execution time of calculate_z_serial_purepython:  0.6803815207999998
The standard dev execution time of calculate_z_serial_purepython:  0.020656422937376365


## Task 1.3 Profile the Julia set code with cProfile and line_profiler the computation.

Measure the time overhead of profiler:

without profiler: 4.2s

cProfile: 5.5s - 4.2s = 1.3s

line_profiler: 30.1s - 4.2s = 25.9s


In [8]:
!python -m cProfile -s cumulative JuliaSet.py

Length of x: 100
Total elements: 10000
@timefn: calculate_z_serial_purepython took 0.048259916 seconds
@timefn: calc_pure_python took 0.051671125 seconds
         364792 function calls (364785 primitive calls) in 0.052 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000    0.052    0.052 {built-in method builtins.exec}
        1    0.000    0.000    0.052    0.052 JuliaSet.py:1(<module>)
      2/1    0.000    0.000    0.052    0.052 JuliaSet.py:11(measure_time)
        1    0.003    0.003    0.052    0.052 JuliaSet.py:23(calc_pure_python)
        1    0.038    0.038    0.048    0.048 JuliaSet.py:64(calculate_z_serial_purepython)
   344236    0.010    0.000    0.010    0.000 {built-in method builtins.abs}
    20200    0.001    0.000    0.001    0.000 {method 'append' of 'list' objects}
      2/1    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1002(_find_and_load)
      2/1    0.000

In [9]:
!python -m cProfile -o profile.stats JuliaSet.py

Length of x: 100
Total elements: 10000
@timefn: calculate_z_serial_purepython took 0.047174917 seconds
@timefn: calc_pure_python took 0.050659582999999994 seconds


In [10]:

!python -m snakeviz profile.stats --server

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fruimins%2FDownloads%2FFDD2356%2FAss1%2Fprofile.stats
^C

Bye!


In [9]:
!python -m kernprof -l JuliaSet.py

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 28.762626209 seconds
@timefn: calc_pure_python took 29.76298125 seconds
Wrote profile results to JuliaSet.py.lprof
Inspect results with:
python -m line_profiler -rmt "JuliaSet.py.lprof"


In [10]:
!python -m line_profiler JuliaSet.py.lprof

Timer unit: 1e-06 s

Total time: 29.3396 s
File: JuliaSet.py
Function: calc_pure_python at line 23

Line #      Hits         Time  Per Hit   % Time  Line Contents
    23                                           @timefn
    24                                           @profile
    25                                           def calc_pure_python(desired_width, max_iterations):
    26                                               """Create a list of complex coordinates (zs) and complex parameters (cs),
    27                                               build Julia set"""
    28         1          1.0      1.0      0.0      x_step = (x2 - x1) / desired_width
    29         1          0.0      0.0      0.0      y_step = (y1 - y2) / desired_width
    30         1          1.0      1.0      0.0      x = []
    31         1          0.0      0.0      0.0      y = []
    32         1          0.0      0.0      0.0      ycoord = y2
    33      1001        125.0      0.1      0.0      while y

In [11]:
!python JuliaSet.py

Length of x: 1000
Total elements: 1000000
@timefn: calculate_z_serial_purepython took 3.652779625 seconds
@timefn: calc_pure_python took 3.872810417 seconds


## Task 1.4 Memory-profile the Juliaset code. Use the memory_profiler and mprof to profile the computation in JuliaSet code. 

reduce the problem size to 100x100

Measure the overhead:

without mem profiler: 0.3s

memory_profiler: 15.0s -0.3s = 14.7s

mprof: 0.8s -0.3s = 0.5s

In [24]:
!python JuliaSet.py

Length of x: 100
Total elements: 10000
@timefn: calculate_z_serial_purepython took 0.036262249999999996 seconds
@timefn: calc_pure_python took 0.038452792 seconds


In [13]:
!python -m memory_profiler JuliaSet.py

Length of x: 100
Total elements: 10000
@timefn: calculate_z_serial_purepython took 13.924193667 seconds
@timefn: calc_pure_python took 14.334788083000001 seconds
Filename: JuliaSet.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    23   45.391 MiB   45.391 MiB           1   @timefn
    24                                         @profile
    25                                         def calc_pure_python(desired_width, max_iterations):
    26                                             """Create a list of complex coordinates (zs) and complex parameters (cs),
    27                                             build Julia set"""
    28   45.391 MiB    0.000 MiB           1       x_step = (x2 - x1) / desired_width
    29   45.391 MiB    0.000 MiB           1       y_step = (y1 - y2) / desired_width
    30   45.391 MiB    0.000 MiB           1       x = []
    31   45.391 MiB    0.000 MiB           1       y = []
    32   45.391 MiB    0.000 MiB           1       ycoord =

In [14]:
!python -m mprof run JuliaSet.py

mprof.py: Sampling memory every 0.1s
running new process
running as a Python program...
Length of x: 100
Total elements: 10000
@timefn: calculate_z_serial_purepython took 0.03536866699999999 seconds
@timefn: calc_pure_python took 0.03769866599999999 seconds


In [22]:
import matplotlib.pyplot as plt
%matplotlib inline
!python -m mprof plot mprofile_20250217101954.dat

Figure(1260x540)


# Exercise II: Profiling Diffusion Process Code 
## Task 2.1 Profile the diffusion code with cProfile and line_profiler the computation.

In [27]:
!python -m cProfile -s cumulative Diffusion.py

         105 function calls in 7.455 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    7.455    7.455 {built-in method builtins.exec}
        1    0.002    0.002    7.455    7.455 Diffusion.py:1(<module>)
        1    0.090    0.090    7.453    7.453 Diffusion.py:18(run_experiment)
       50    7.351    0.147    7.363    0.147 Diffusion.py:4(evolve)
       50    0.012    0.000    0.012    0.000 Diffusion.py:6(<listcomp>)
        1    0.001    0.001    0.001    0.001 Diffusion.py:21(<listcomp>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [30]:
!python -m cProfile -o profile2.stats Diffusion.py

In [31]:
!python -m snakeviz profile2.stats --server

snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fruimins%2FDownloads%2FFDD2356%2Fprofile2.stats
^C

Bye!


In [36]:
!python -m kernprof -l Diffusion.py
!python -m line_profiler Diffusion.py.lprof

Wrote profile results to Diffusion.py.lprof
Inspect results with:
python -m line_profiler -rmt "Diffusion.py.lprof"
Timer unit: 1e-06 s

Total time: 23.2777 s
File: Diffusion.py
Function: evolve at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           @profile
     4                                           def evolve(grid, dt, D=1.0):
     5        50         32.0      0.6      0.0      xmax, ymax = grid_shape
     6        50      12507.0    250.1      0.1      new_grid = [[0.0] * ymax for x in range(xmax)]
     7     32050       4504.0      0.1      0.0      for i in range(xmax):
     8  20512000    2458362.0      0.1     10.6          for j in range(ymax):
     9  20480000    1923732.0      0.1      8.3              grid_xx = (
    10  20480000    6459402.0      0.3     27.7                  grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    11                                                   

## Task 2.2 Memory-profile the diffusion code. Use the memory_profiler and mprof to profile the computation. 

In [38]:
!python -m memory_profiler Diffusion.py

Filename: Diffusion.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     3   49.859 MiB 1582.016 MiB          50   @profile
     4                                         def evolve(grid, dt, D=1.0):
     5   49.859 MiB -908.141 MiB          50       xmax, ymax = grid_shape
     6   53.016 MiB -645325.391 MiB       32150       new_grid = [[0.0] * ymax for x in range(xmax)]
     7   61.547 MiB -921561.984 MiB       32050       for i in range(xmax):
     8   61.547 MiB -590123916.297 MiB    20512000           for j in range(ymax):
     9   61.547 MiB -589202749.062 MiB    20480000               grid_xx = (
    10   61.547 MiB -589203015.250 MiB    20480000                   grid[(i + 1) % xmax][j] + grid[(i - 1) % xmax][j] - 2.0 * grid[i][j]
    11                                                     )
    12   61.547 MiB -589203004.781 MiB    20480000               grid_yy = (
    13   61.547 MiB -589202883.328 MiB    20480000                   grid[i][(j + 1) % ymax] +

In [40]:
!python -m mprof run Diffusion.py
import matplotlib.pyplot as plt
%matplotlib inline
#!python -m mprof plot mprofile_20250217101954.dat

mprof.py: Sampling memory every 0.1s
running new process
running as a Python program...
